##### Importar módulos de implementação dos algoritmos

In [1]:
from models import *
from utils import *

##### Carrega os dados tratados

In [2]:
gun = pd.read_csv("../Datasets/Tratados/GunControl_400.csv", low_memory=False).fillna("").iloc[:, 10:]
gay = pd.read_csv("../Datasets/Tratados/GayMarriage_400.csv", low_memory=False).fillna("").iloc[:, 10:]
rest = pd.read_csv("../Datasets/Tratados/ReviewAMT_500_t.csv", low_memory=False).fillna("").iloc[:, 11:]

# Metodologia padrão (LOO)

##### Iteração entre Qtd de Usuários (até 200)

In [3]:
for qtd_users_p_base in range(5,101,5):

    qtd_users = qtd_users_p_base * 2
    print("=========== Treinando/Validando modelos utilizando", qtd_users, "usuários ===========")
    print("+++ Preparando dados +++")
          
    n = qtd_users_p_base * 4
    # dados = pd.concat([gun.iloc[:n, :], gay.iloc[:n, :], rest.iloc[:n, :]], axis=0, sort=False).fillna("").reset_index(drop=True)
    dados = pd.concat([gun.iloc[:n, :], gay.iloc[:n, :]], axis=0, sort=False).fillna("").reset_index(drop=True)

    print("+++ Executando k-fold +++")
    # Cria folds de usuários e amostras
    user_list = dados.User_Label
    sample_list = dados.Sample_Label
    interno, externo = k_usuarios(user_list, n_folds=5)
    kf = k_amostras(interno, externo, pd.concat([user_list, sample_list], axis=1), method="LOO") # LOO, DifferentTexts, DifferentPatterns


    ####### Execução treinamentos e validações
    ### Mod1
    print("+++ Executando técnica 1 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 1.1}
    mod1 = modelo(1, dados, kf, param)


    ### Mod2
    print("+++ Executando técnica 2 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 0.5}

    mod2 = modelo(2, dados, kf, param)


    ### Mod3
    print("+++ Executando técnica 3 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 1.1}

    mod21 = modelo(2.1, dados, kf, param)


    ### Mod4
    print("+++ Executando técnica 4 +++")
    # Primeiro com "new" para extrair variáveis
    param = {"action": "new",     # new, load
             "algorithm" : "tree",        # tree, svm
             "decision" : "merge",        # binary, multiclass, merge
             "th" : 0}                # range(0,1)
    mod3 = modelo(3, dados, kf, param)


    # Itera entre parametros
    mods3 = []
    for i in ["tree", "svm"]:
        for j in ["merge"]:

            param = {"action": "load",     # new, load
                     "algorithm" : i,        # tree, svm
                     "decision" : j,        # binary, multiclass, merge
                     "th" : 0}                # range(0,1)

            mod3 = modelo(3, dados, kf, param)
            mods3.append([mod3, i, j])


    ### Mod5
    print("+++ Executando técnica 5 +++")
    # Primeiro com "new" para extrair variáveis
    tree_params = {"criterion" : 'gini',
                   "splitter" : 'best',
                   "max_depth" : 10,
                   "min_samples_split" : 2, 
                   "min_samples_leaf" : 2, 
                   "min_weight_fraction_leaf" : 0.0,
                   "max_features" : 10, 
                   "random_state" : 0,
                   "max_leaf_nodes" : None,
                   "min_impurity_decrease" : 0.0,
                   "min_impurity_split" : None,
                   "class_weight" : None,
                   "presort" : False}
    param = {"action": "new",               # new, load
             "algorithm" : "tree",           # tree, svm
             "decision" : "merge",           # binary, multiclass, merge
             "model_params" : tree_params}       
    mod4 = modelo(4, dados, kf, param)

    mods4 = []
    for i in ["tree", "svm"]:
        for j in ["merge"]:
            tree_params = {"criterion" : 'gini',
                           "splitter" : 'best',
                           "max_depth" : 10,
                           "min_samples_split" : 2,
                           "min_samples_leaf" : 2,
                           "min_weight_fraction_leaf" : 0.0,
                           "max_features" : 10,
                           "random_state" : 0,
                           "max_leaf_nodes" : None,
                           "min_impurity_decrease" : 0.0,
                           "min_impurity_split" : None,
                           "class_weight" : None,
                           "presort" : False}

            param = {"action": "load",               # new, load
                     "algorithm" : i,           # tree, svm
                     "decision" : j,                 # binary, multiclass, merge
                     "model_params" : tree_params}       

            mod4 = modelo(4, dados, kf, param)
            mods4.append([mod4, i, j])


    ###### Salva resultados dos modelos
    print("+++ Salvaldo Resultados +++\n\n")        
    var_to_pickle(mod1, "../Caches/results_final_mod1_" + str(qtd_users) + "users")
    var_to_pickle(mod2, "../Caches/results_final_mod2_" + str(qtd_users) + "users")
    var_to_pickle(mod21, "../Caches/results_final_mod21_" + str(qtd_users) + "users")
    var_to_pickle(mods3, "../Caches/results_final_mod3_" + str(qtd_users) + "users")
    var_to_pickle(mods4, "../Caches/results_final_mod4_" + str(qtd_users) + "users")

=========== Treinando/Validando modelos utilizando 10 usuários ===========
+++ Preparando dados +++
+++ Executando k-fold +++
+++ Executando técnica 1 +++



+++ Executando técnica 2 +++



+++ Executando técnica 3 +++



+++ Executando técnica 4 +++



+++ Executando técnica 5 +++


...


+++ Salvaldo Resultados +++


=========== Treinando/Validando modelos utilizando 120 usuários ===========
+++ Preparando dados +++
+++ Executando k-fold +++
+++ Executando técnica 1 +++



+++ Executando técnica 2 +++



+++ Executando técnica 3 +++



+++ Executando técnica 4 +++


KeyboardInterrupt: 

##### Iteração entre Qtd de Usuários (até 1200, técnicas 1,2,3)

In [ ]:
for qtd_users_p_base in range(5,600,5):

    qtd_users = qtd_users_p_base * 2
    print("=========== Treinando/Validando modelos utilizando", qtd_users, "usuários ===========")
    print("+++ Preparando dados +++")
          
    n = qtd_users_p_base * 4
    # dados = pd.concat([gun.iloc[:n, :], gay.iloc[:n, :], rest.iloc[:n, :]], axis=0, sort=False).fillna("").reset_index(drop=True)
    dados = pd.concat([gun.iloc[:n, :], gay.iloc[:n, :]], axis=0, sort=False).fillna("").reset_index(drop=True)

    print("+++ Executando k-fold +++")
    # Cria folds de usuários e amostras
    user_list = dados.User_Label
    sample_list = dados.Sample_Label
    interno, externo = k_usuarios(user_list, n_folds=5)
    kf = k_amostras(interno, externo, pd.concat([user_list, sample_list], axis=1), method="LOO") # LOO, DifferentTexts, DifferentPatterns


    ####### Execução treinamentos e validações
    ### Mod1
    print("+++ Executando técnica 1 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 1.1}
    mod1 = modelo(1, dados, kf, param)


    ### Mod2
    print("+++ Executando técnica 2 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 0.5}

    mod2 = modelo(2, dados, kf, param)


    ### Mod3
    print("+++ Executando técnica 3 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 1.1}

    mod21 = modelo(2.1, dados, kf, param)

    ###### Salva resultados dos modelos
    print("+++ Salvaldo Resultados +++\n\n")        
    var_to_pickle(mod1, "../Caches/results_final_mod1_" + str(qtd_users) + "users")
    var_to_pickle(mod2, "../Caches/results_final_mod2_" + str(qtd_users) + "users")
    var_to_pickle(mod21, "../Caches/results_final_mod21_" + str(qtd_users) + "users")

# Análises adicionais

##### Diferentes textos - verdadeiro/falso

In [ ]:
for qtd_users_p_base in [10, 100]:

    qtd_users = qtd_users_p_base * 2
    print("=========== Treinando/Validando modelos utilizando", qtd_users, "usuários ===========")
    print("+++ Preparando dados +++")
          
    n = qtd_users_p_base * 4
    # dados = pd.concat([gun.iloc[:n, :], gay.iloc[:n, :], rest.iloc[:n, :]], axis=0, sort=False).fillna("").reset_index(drop=True)
    dados = pd.concat([gun.iloc[:n, :], gay.iloc[:n, :]], axis=0, sort=False).fillna("").reset_index(drop=True)

    print("+++ Executando k-fold +++")
    # Cria folds de usuários e amostras
    user_list = dados.User_Label
    sample_list = dados.Sample_Label
    interno, externo = k_usuarios(user_list, n_folds=5)
    kf = k_amostras(interno, externo, pd.concat([user_list, sample_list], axis=1), method="DifferentTexts") # LOO, DifferentTexts, DifferentPatterns


    ####### Execução treinamentos e validações
    ### Mod1
    print("+++ Executando técnica 1 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 1.1}
    mod1 = modelo(1, dados, kf, param)


    ### Mod2
    print("+++ Executando técnica 2 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 0.5}

    mod2 = modelo(2, dados, kf, param)


    ### Mod3
    print("+++ Executando técnica 3 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 1.1}

    mod21 = modelo(2.1, dados, kf, param)


    ### Mod4
    print("+++ Executando técnica 4 +++")
    # Primeiro com "new" para extrair variáveis
    param = {"action": "new",     # new, load
             "algorithm" : "tree",        # tree, svm
             "decision" : "merge",        # binary, multiclass, merge
             "th" : 0}                # range(0,1)
    mod3 = modelo(3, dados, kf, param)


    # Itera entre parametros
    mods3 = []
    for i in ["tree", "svm"]:
        for j in ["merge"]:

            param = {"action": "load",     # new, load
                     "algorithm" : i,        # tree, svm
                     "decision" : j,        # binary, multiclass, merge
                     "th" : 0}                # range(0,1)

            mod3 = modelo(3, dados, kf, param)
            mods3.append([mod3, i, j])


    ### Mod5
    print("+++ Executando técnica 5 +++")
    # Primeiro com "new" para extrair variáveis
    tree_params = {"criterion" : 'gini',
                   "splitter" : 'best',
                   "max_depth" : 10,
                   "min_samples_split" : 2, 
                   "min_samples_leaf" : 2, 
                   "min_weight_fraction_leaf" : 0.0,
                   "max_features" : 10, 
                   "random_state" : 0,
                   "max_leaf_nodes" : None,
                   "min_impurity_decrease" : 0.0,
                   "min_impurity_split" : None,
                   "class_weight" : None,
                   "presort" : False}
    param = {"action": "new",               # new, load
             "algorithm" : "tree",           # tree, svm
             "decision" : "merge",           # binary, multiclass, merge
             "model_params" : tree_params}       
    mod4 = modelo(4, dados, kf, param)

    mods4 = []
    for i in ["tree", "svm"]:
        for j in ["merge"]:

            tree_params = {"criterion" : 'gini',
                           "splitter" : 'best',
                           "max_depth" : 10,
                           "min_samples_split" : 2,
                           "min_samples_leaf" : 2,
                           "min_weight_fraction_leaf" : 0.0,
                           "max_features" : 10,
                           "random_state" : 0,
                           "max_leaf_nodes" : None,
                           "min_impurity_decrease" : 0.0,
                           "min_impurity_split" : None,
                           "class_weight" : None,
                           "presort" : False}

            param = {"action": "load",               # new, load
                     "algorithm" : i,           # tree, svm
                     "decision" : j,                 # binary, multiclass, merge
                     "model_params" : tree_params}       

            mod4 = modelo(4, dados, kf, param)
            mods4.append([mod4, i, j])


    ###### Salva resultados dos modelos
    print("+++ Salvaldo Resultados +++\n\n")        
    var_to_pickle(mod1, "../Caches/results_final_mod1_" + str(qtd_users) + "users_DT")
    var_to_pickle(mod2, "../Caches/results_final_mod2_" + str(qtd_users) + "users_DT")
    var_to_pickle(mod21, "../Caches/results_final_mod21_" + str(qtd_users) + "users_DT")
    var_to_pickle(mods3, "../Caches/results_final_mod3_" + str(qtd_users) + "users_DT")
    var_to_pickle(mods4, "../Caches/results_final_mod4_" + str(qtd_users) + "users_DT")

##### Diferentes padrões - original/cópia

In [ ]:
for qtd_users_p_base in [10, 100]:

    qtd_users = qtd_users_p_base * 2
    print("=========== Treinando/Validando modelos utilizando", qtd_users, "usuários ===========")
    print("+++ Preparando dados +++")
          
    n = qtd_users_p_base * 4
    # dados = pd.concat([gun.iloc[:n, :], gay.iloc[:n, :], rest.iloc[:n, :]], axis=0, sort=False).fillna("").reset_index(drop=True)
    dados = pd.concat([gun.iloc[:n, :], gay.iloc[:n, :]], axis=0, sort=False).fillna("").reset_index(drop=True)

    print("+++ Executando k-fold +++")
    # Cria folds de usuários e amostras
    user_list = dados.User_Label
    sample_list = dados.Sample_Label
    interno, externo = k_usuarios(user_list, n_folds=5)
    kf = k_amostras(interno, externo, pd.concat([user_list, sample_list], axis=1), method="DifferentPatterns") # LOO, DifferentTexts, DifferentPatterns


    ####### Execução treinamentos e validações
    ### Mod1
    print("+++ Executando técnica 1 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 1.1}
    mod1 = modelo(1, dados, kf, param)


    ### Mod2
    print("+++ Executando técnica 2 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 0.5}

    mod2 = modelo(2, dados, kf, param)


    ### Mod3
    print("+++ Executando técnica 3 +++")
    param = {"dist_R" : [2],
             "dist_A" : [2],
             "t" : 1.2,
             "k": 1.1}

    mod21 = modelo(2.1, dados, kf, param)


    ### Mod4
    print("+++ Executando técnica 4 +++")
    # Primeiro com "new" para extrair variáveis
    param = {"action": "new",     # new, load
             "algorithm" : "tree",        # tree, svm
             "decision" : "merge",        # binary, multiclass, merge
             "th" : 0}                # range(0,1)
    mod3 = modelo(3, dados, kf, param)


    # Itera entre parametros
    mods3 = []
    for i in ["tree", "svm"]:
        for j in ["merge"]:

            param = {"action": "load",     # new, load
                     "algorithm" : i,        # tree, svm
                     "decision" : j,        # binary, multiclass, merge
                     "th" : 0}                # range(0,1)

            mod3 = modelo(3, dados, kf, param)
            mods3.append([mod3, i, j])


    ### Mod5
    print("+++ Executando técnica 5 +++")
    # Primeiro com "new" para extrair variáveis
    tree_params = {"criterion" : 'gini',
                   "splitter" : 'best',
                   "max_depth" : 10,
                   "min_samples_split" : 2, 
                   "min_samples_leaf" : 2, 
                   "min_weight_fraction_leaf" : 0.0,
                   "max_features" : 10, 
                   "random_state" : 0,
                   "max_leaf_nodes" : None,
                   "min_impurity_decrease" : 0.0,
                   "min_impurity_split" : None,
                   "class_weight" : None,
                   "presort" : False}
    param = {"action": "new",               # new, load
             "algorithm" : "tree",           # tree, svm
             "decision" : "merge",           # binary, multiclass, merge
             "model_params" : tree_params}       
    mod4 = modelo(4, dados, kf, param)

    mods4 = []
    for i in ["tree", "svm"]:
        for j in ["merge"]:

            tree_params = {"criterion" : 'gini',
                           "splitter" : 'best',
                           "max_depth" : 10,
                           "min_samples_split" : 2,
                           "min_samples_leaf" : 2,
                           "min_weight_fraction_leaf" : 0.0,
                           "max_features" : 10,
                           "random_state" : 0,
                           "max_leaf_nodes" : None,
                           "min_impurity_decrease" : 0.0,
                           "min_impurity_split" : None,
                           "class_weight" : None,
                           "presort" : False}

            param = {"action": "load",               # new, load
                     "algorithm" : i,           # tree, svm
                     "decision" : j,                 # binary, multiclass, merge
                     "model_params" : tree_params}       

            mod4 = modelo(4, dados, kf, param)
            mods4.append([mod4, i, j])


    ###### Salva resultados dos modelos
    print("+++ Salvaldo Resultados +++\n\n")        
    var_to_pickle(mod1, "../Caches/results_final_mod1_" + str(qtd_users) + "users_DP")
    var_to_pickle(mod2, "../Caches/results_final_mod2_" + str(qtd_users) + "users_DP")
    var_to_pickle(mod21, "../Caches/results_final_mod21_" + str(qtd_users) + "users_DP")
    var_to_pickle(mods3, "../Caches/results_final_mod3_" + str(qtd_users) + "users_DP")
    var_to_pickle(mods4, "../Caches/results_final_mod4_" + str(qtd_users) + "users_DP")